In [1]:
import os
from pathlib import Path
import pandas as pd
import datetime as dt
import math
import pynmea2
from io import StringIO
from datetime import timedelta

In [2]:
def decdeg2nmea(dd, type_coord):
    dd = float(dd)
    num = abs(dd)
    d = float(math.floor(num))
    m = num - d
    sign = '-' if dd < 0 else ''
    if type_coord =='lat':
        return '%03i%08.5f' % (int(d), m * 60.00)
    elif type_coord =='lon':
        return '%02i%08.5f' % (int(d), m * 60.00)
    
#   return sign+'%03i%02.5f' % (int(d), m * 60.00)

In [3]:
# folder_path = os.getcwd()
# print(folder_path)

In [4]:
# Specify the folder containing .csv files
folder_path = ''
print(folder_path)

s3//


In [5]:
def process_csv_files(folder_path):
    # Iterate through sub-folders and files in the main folder
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.csv'):
                csv_file_path = os.path.join(root, file_name)
                log_file_path = os.path.join(root, os.path.splitext(file_name)[0] + '.LOG')

                try:
                    # Read the CSV file into a DataFrame
                    df = pd.read_csv(csv_file_path)
                    df.rename(columns={'GPS (Lat.) [deg]': 'latitude', 'GPS (Long.) [deg]': 'longitude'}, inplace=True)
                    df['%'] = None
                    df['GPST'] = None
                    df['timestamp'] = list(
                        map(lambda x, y, z: dt.datetime.strptime(f'{x} {y}', '%d/%m/%Y %H:%M:%S.%f')
                            if pd.notnull(x)
                            else (dt.datetime.strptime(str(z), '%Y-%m-%dT%H:%M:%S.%fZ')  # with timezone
                                  if str(z) != 'nan' else None), df['%'], df.GPST, df.date))
                    hour_correction = 3
                    df['timestamp'] = pd.to_datetime(df['timestamp']) - timedelta(hours=hour_correction)
                    df['timestamp_str'] = list(map(lambda x: dt.datetime.strftime(x, '%d-%m-%Y %H:%M:%S'), df.timestamp))
                    df['DateTimeNumbers'] = list(
                        map(lambda x: float((x.hour * 10000) + (x.minute * 100) + x.second), df.timestamp))
                    df['latitude_nmea'] = list(map(lambda x: decdeg2nmea(x, type_coord='lat'), df.latitude.values))
                    df['longitude_nmea'] = list(map(lambda x: decdeg2nmea(x, type_coord='lon'), df.longitude.values))
                    df['N/S'] = ''
                    df['E/W'] = ''

                    for index, row in df.iterrows():
                        # Check the sign of latitude
                        if row['latitude'] >= 0:
                            df.at[index, 'N/S'] = 'N'
                        else:
                            df.at[index, 'N/S'] = 'S'

                        # Check the sign of longitude
                        if row['longitude'] >= 0:
                            df.at[index, 'E/W'] = 'E'
                        else:
                            df.at[index, 'E/W'] = 'W'

                    df['GPGGA'] = list(
                        map(lambda x: '$GPGGA,{0:.4f},{1},{2},{3},{4},1,0,,,M,,M,,\n'.format(x[0], x[1], x[2], x[3], x[4]),
                            df[['DateTimeNumbers', 'latitude_nmea', 'N/S', 'longitude_nmea', 'E/W']].values))
                    df['GPRMC'] = list(
                        map(lambda x: '$GPRMC,{0:.4f},A,{1},{2},{3},{4},19.31,,250620,,,\n'.format(x[0], x[1], x[2], x[3], x[4]),
                            df[['DateTimeNumbers', 'latitude_nmea', 'N/S', 'longitude_nmea', 'E/W']].values))
                    df['lines'] = df.GPGGA.values + df.GPRMC.values
                    PRE_TEXT = '@Sonygps/ver5.0/wgs-84/20200625135753.000/\n@Sonygpsoption/0/20200625135754.000/20200625135754.000/\n'

                    with open(log_file_path, "x") as file:
                        file.write(PRE_TEXT)
                        for line in df.lines.values:
                            file.write(line)
                    print(f"Converted '{csv_file_path}' to '{log_file_path}'")

                except Exception as e:
                    print(f"Skipping '{csv_file_path}': {str(e)}")

In [6]:
process_csv_files(folder_path)

Converted 's3//GX011381.csv' to 's3//GX011381.LOG'
Converted 's3//GX011382.csv' to 's3//GX011382.LOG'
Converted 's3//GX011383.csv' to 's3//GX011383.LOG'
Converted 's3//GX011384.csv' to 's3//GX011384.LOG'
Converted 's3//GX011385.csv' to 's3//GX011385.LOG'
Converted 's3//GX011386.csv' to 's3//GX011386.LOG'
Converted 's3//GX011387.csv' to 's3//GX011387.LOG'
Converted 's3//GX011387_1.csv' to 's3//GX011387_1.LOG'
Converted 's3//GX011388.csv' to 's3//GX011388.LOG'
Converted 's3//GX011388_1.csv' to 's3//GX011388_1.LOG'
Converted 's3//GX011389.csv' to 's3//GX011389.LOG'
Converted 's3//GX011389_1.csv' to 's3//GX011389_1.LOG'
Converted 's3//GX011390.csv' to 's3//GX011390.LOG'
Converted 's3//GX011390_1.csv' to 's3//GX011390_1.LOG'
Converted 's3//GX011391.csv' to 's3//GX011391.LOG'
Converted 's3//GX011391_1.csv' to 's3//GX011391_1.LOG'
Converted 's3//GX011392.csv' to 's3//GX011392.LOG'
Converted 's3//GX011392_1.csv' to 's3//GX011392_1.LOG'
Converted 's3//GX011392_2.csv' to 's3//GX011392_2.LOG'
Con